<a href="https://colab.research.google.com/github/dtd1111/test2/blob/main/share_StS_run_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. 메뉴바에서 파일->코드를 사본 저장 하기

#1. 우상단 연결버튼 클릭

#2. 파일(좌측 폴터 모양 버튼 클릭)에 run파일을 zip으로 압축 후 업로드

#3. 메뉴바에서 런타임-모두실행

#4. 파일에서 .xlsx파일을 다운로드(없으면 새로고침 눌러볼 것)

In [ ]:
#@title Module import
import json
import sys
import pandas as pd

from tqdm.auto import tqdm
from collections import Counter
from datetime import datetime

from os import listdir
from os.path import getmtime,isfile, join
from openpyxl import Workbook
from openpyxl.styles import PatternFill# @title 기본 제목 텍스트

In [ ]:
#@title 사전작업
#액셀파일 생성
write_wb = Workbook()

#요약 시트 사전 작업
write_summary = write_wb.create_sheet('요약',0)
write_summary.append(["캐릭터","니오우 패널티","니오우 보너스","도달 층수", "승/패", "패배 원인", "덱 매수", "유물 수량","1막 보스유물","2막 보스유물", "플레이 시간", "런 일시", "점수"])
write_summary.freeze_panes = 'A2'

In [ ]:
#@title 요약 시트 관련 함수
def check_boss_relic(json_object):
  chosen_boss_relic = [None,None]
  #1막 보스유물
  if json_object['floor_reached'] >= 17:
    chosen_boss_relic[0] = json_object['boss_relics'][0]['picked']

  #2막 보스유물
  if json_object['floor_reached'] >= 34:
    chosen_boss_relic[1] = json_object['boss_relics'][1]['picked']
  return chosen_boss_relic

def record_summary(json_object):
  run_format = {}

  #캐릭터
  run_format["character"] = json_object['character_chosen']

  #고래밥 패널티
  if json_object['neow_cost'] == "NONE":
    run_format["neow_cost"] = None
  else:
    run_format["neow_cost"] = json_object['neow_cost']

  #고래밥 보너스
  run_format["neow_bonus"] = json_object['neow_bonus']

  #도달 층수
  run_format["floor_reached"] = json_object['floor_reached']

  #승패
  if json_object['victory']:
    run_format["is_win"] = "승리"
  else:
    run_format["is_win"] = "패배"

  #사망 원인
  killed_by = None
  if not json_object['victory']:
      try:
        killed_by = json_object["killed_by"]
      except KeyError:
        killed_by = "Unknown"
  else:
    killed_by = None
  run_format["killed_by"] = killed_by

  #덱
  run_format["deck_length"] = len(json_object['master_deck'])
  #run_format["master_deck"] = json_object['master_deck']

  #유물
  run_format["relics_collected"] = len(json_object['relics'])
  #run_format["relics"] = json_object['relics']

  #보스 유물
  run_format["act1_boss_relic"] = check_boss_relic(json_object)[0]
  run_format["act2_boss_relic"] = check_boss_relic(json_object)[1]

  #런 지속 시간
  run_format["playtime"] = json_object["playtime"]
  run_format["local_time"] = datetime.strptime(json_object["local_time"], '%Y%m%d%H%M%S')
  run_format["score"] = json_object["score"]

  # Define the fill color (e.g., red for 'Fail')
  red_fill = PatternFill(start_color="F5A9A9", end_color="F5A9A9", fill_type="solid")
  green_fill = PatternFill(start_color="81F781", end_color="81F781", fill_type="solid")

  write_summary.append(list(run_format.values()))

  for row in write_summary.iter_rows(min_row=2, max_row=write_summary.max_row, max_col=write_summary.max_column):
      status_cell = row[4]  # 승패 행의 승리/패배를 참고
      if status_cell.value == '패배':
          for cell in row:
              cell.fill = red_fill

      elif status_cell.value == '승리':
          for cell in row:
              cell.fill = green_fill

In [ ]:
#@title 카드 시트 관련 함수
def record_cards(total_count,character,nums,total_games):
  #요약 시트 사전 작업
  write_cards = write_wb.create_sheet(f'{character}카드',1+2*nums)
  write_cards.append(["카드 이름", "포함 횟수", "포함 승리횟수", "포함 패배횟수", "포함 승리율", "포함 패배율"])
  write_cards.freeze_panes = 'A2'

  for key, value in sorted(total_count.items()):
    print(key, ":", value)

    write_cards.append([key, (value[0]+value[1]),
                      value[0], value[1],
                      value[0]/(value[0]+value[1]), value[1]/(value[0]+value[1])])

    # chose_rate = (value[0]+value[1])/total_games
    # write_cards.append([key,f"={(value[0]+value[1])}/{total_games}",
    #                     f"={value[0]}/{(value[0]+value[1])}",f"={value[1]}/{(value[0]+value[1])}"])


In [ ]:
#@title 유물 시트 관련 함수
def record_relics(total_count,character,nums,total_games):
  write_relics = write_wb.create_sheet(f'{character}유물',2+2*nums)
  write_relics.append(["유물 이름", "포함 횟수", "포함 승리횟수", "포함 패배횟수", "포함 승리율", "포함 패배율"])
  write_relics.freeze_panes = 'A2'

  for key, value in sorted(total_count.items()):
    print(key, ":", value)

    write_relics.append([key, (value[0]+value[1]),
                      value[0], value[1],
                      value[0]/(value[0]+value[1]), value[1]/(value[0]+value[1])])

In [ ]:
# @title 메인 작업
#@파일 개봉
!unzip -o runs.zip
files_list = ["IRONCLAD","THE_SILENT","DEFECT","WATCHER"]

#캐릭터 단위로 순회
for num,i in enumerate(files_list):
  directory_path = f"./{i}"
  onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
  onlyfiles.sort(key=lambda x: getmtime(join(directory_path, x)),reverse=True)

  win_deck_counter = Counter()
  lose_deck_counter = Counter()
  total_cards_count = {}
  total_relics_count = {}

  for j,file in enumerate(tqdm(onlyfiles,desc=f"{i} 데이터 정리중"),start=1):
  #for j,file in enumerate(onlyfiles,start=1):
    #run파일 로드
    try:
      with open(f"{directory_path}/{file}", encoding="cp949") as f:
        #run_format["format"] = "cp949"
        json_object = json.load(f)

    except UnicodeDecodeError as e:
      #로드 오류시 UTF8로 재시도
      with open(f"{directory_path}/{file}", encoding="UTF8") as f:
        #run_format["format"] = "UTF8"
        json_object = json.load(f)

    record_summary(json_object)
    cards_contained = []
    relics_contained = []

    #카드 기록
    deck_count = Counter(json_object['master_deck'])
    for key in deck_count:
      if '+' in key:
        key = key.split('+')[0]
      cards_contained.append(key)
    cards_contained = list(set(cards_contained))

    for key in cards_contained:
      if not key in total_cards_count:
        total_cards_count[key] = [0,0]

      if json_object['victory']:
        total_cards_count[key][0] += 1
      else:
        total_cards_count[key][1] += 1

    #유물 기록
    relics_contained = json_object['relics']
    for key in relics_contained:
      if not key in total_relics_count:
        total_relics_count[key] = [0,0]

      if json_object['victory']:
        total_relics_count[key][0] += 1
      else:
        total_relics_count[key][1] += 1

  record_cards(total_cards_count,i,num,j)
  record_relics(total_relics_count,i,num,j)

Archive:  runs.zip
   creating: DEFECT/
  inflating: DEFECT/1714987675.run   
  inflating: DEFECT/1725803704.run   
   creating: IRONCLAD/
  inflating: IRONCLAD/1713075552.run  
  inflating: IRONCLAD/1713078937.run  
  inflating: IRONCLAD/1713083066.run  
  inflating: IRONCLAD/1713094543.run  
  inflating: IRONCLAD/1713188601.run  
  inflating: IRONCLAD/1713494097.run  
  inflating: IRONCLAD/1714520948.run  
  inflating: IRONCLAD/1714909870.run  
  inflating: IRONCLAD/1714990168.run  
  inflating: IRONCLAD/1714990690.run  
  inflating: IRONCLAD/1714994033.run  
  inflating: IRONCLAD/1715067513.run  
  inflating: IRONCLAD/1715070371.run  
  inflating: IRONCLAD/1715166015.run  
  inflating: IRONCLAD/1715169894.run  
  inflating: IRONCLAD/1715330122.run  
  inflating: IRONCLAD/1715414692.run  
  inflating: IRONCLAD/1715514638.run  
  inflating: IRONCLAD/1715588326.run  
  inflating: IRONCLAD/1715675266.run  
  inflating: IRONCLAD/1715774828.run  
  inflating: IRONCLAD/1715779098.run  
  i

IRONCLAD 데이터 정리중:   0%|          | 0/140 [00:00<?, ?it/s]

Anger : [19, 16]
Apotheosis : [18, 2]
Armaments : [42, 21]
AscendersBane : [84, 56]
Bandage Up : [2, 0]
Barricade : [34, 8]
Bash : [71, 54]
Battle Trance : [41, 12]
Berserk : [8, 1]
Bite : [6, 2]
Blind : [3, 1]
Blood for Blood : [22, 14]
Bloodletting : [32, 7]
Bludgeon : [5, 6]
Body Slam : [35, 11]
Brutality : [17, 5]
Burning Pact : [41, 10]
Carnage : [15, 7]
Clash : [0, 1]
Cleave : [6, 13]
Clothesline : [15, 10]
Clumsy : [2, 0]
Combust : [12, 7]
Corruption : [35, 7]
CurseOfTheBell : [11, 5]
Dark Embrace : [69, 13]
Dark Shackles : [5, 0]
Decay : [1, 1]
Deep Breath : [4, 0]
Defend_R : [68, 50]
Demon Form : [11, 5]
Disarm : [39, 9]
Discovery : [8, 1]
Double Tap : [12, 5]
Doubt : [0, 1]
Dropkick : [20, 12]
Dual Wield : [23, 4]
Enlightenment : [3, 0]
Entrench : [26, 5]
Evolve : [39, 11]
Exhume : [27, 11]
Feed : [35, 11]
Feel No Pain : [63, 18]
Fiend Fire : [33, 7]
Finesse : [9, 1]
Fire Breathing : [6, 3]
Flame Barrier : [32, 13]
Flash of Steel : [11, 4]
Flex : [9, 6]
Ghostly : [8, 4]
Ghost

THE_SILENT 데이터 정리중:   0%|          | 0/21 [00:00<?, ?it/s]

A Thousand Cuts : [1, 0]
Accuracy : [5, 0]
Acrobatics : [13, 4]
Adrenaline : [7, 2]
After Image : [3, 1]
All Out Attack : [1, 0]
Apotheosis : [2, 0]
AscendersBane : [14, 7]
Backflip : [6, 4]
Backstab : [1, 2]
Bane : [0, 1]
Bite : [1, 0]
Blade Dance : [7, 4]
Blur : [5, 0]
Bouncing Flask : [3, 2]
Bullet Time : [2, 0]
Burst : [3, 0]
Calculated Gamble : [9, 3]
Caltrops : [2, 0]
Catalyst : [3, 0]
Choke : [0, 1]
Cloak And Dagger : [2, 0]
Clumsy : [1, 1]
Concentrate : [6, 0]
Corpse Explosion : [3, 1]
Crippling Poison : [1, 0]
CurseOfTheBell : [1, 0]
Dagger Spray : [3, 1]
Dagger Throw : [5, 3]
Dash : [3, 0]
Deadly Poison : [4, 1]
Deep Breath : [1, 0]
Defend_G : [9, 7]
Deflect : [9, 0]
Die Die Die : [4, 1]
Discovery : [1, 0]
Distraction : [1, 0]
Dodge and Roll : [4, 0]
Doppelganger : [2, 0]
Doubt : [0, 1]
Endless Agony : [2, 0]
Enlightenment : [0, 1]
Envenom : [4, 0]
Escape Plan : [6, 2]
Eviscerate : [8, 2]
Expertise : [1, 0]
Finisher : [2, 2]
Flechettes : [2, 0]
Flying Knee : [1, 0]
Footwork :

DEFECT 데이터 정리중:   0%|          | 0/2 [00:00<?, ?it/s]

AscendersBane : [0, 2]
Ball Lightning : [0, 1]
Beam Cell : [0, 1]
Cold Snap : [0, 1]
Conserve Battery : [0, 1]
Creative AI : [0, 1]
Defend_B : [0, 2]
Dualcast : [0, 2]
Electrodynamics : [0, 1]
Force Field : [0, 1]
Fusion : [0, 1]
Go for the Eyes : [0, 1]
Heatsinks : [0, 1]
Hologram : [0, 1]
Rip and Tear : [0, 1]
Steam : [0, 1]
Strike_B : [0, 2]
Sunder : [0, 2]
Undo : [0, 1]
Zap : [0, 2]
Coffee Dripper : [0, 1]
Cracked Core : [0, 1]
Cursed Key : [0, 1]
Orrery : [0, 1]
Smiling Mask : [0, 1]
TungstenRod : [0, 1]
War Paint : [0, 1]
White Beast Statue : [0, 1]


WATCHER 데이터 정리중:   0%|          | 0/7 [00:00<?, ?it/s]

Adaptation : [2, 0]
AscendersBane : [4, 3]
BattleHymn : [1, 0]
Bite : [0, 1]
Blasphemy : [0, 1]
BowlingBash : [1, 2]
Brilliance : [0, 1]
CarveReality : [0, 2]
ClearTheMind : [1, 0]
Collect : [2, 1]
Conclude : [1, 0]
ConjureBlade : [1, 0]
Consecrate : [2, 2]
CrushJoints : [3, 1]
CurseOfTheBell : [1, 0]
CutThroughFate : [3, 2]
DeceiveReality : [1, 0]
Defend_P : [2, 3]
DeusExMachina : [1, 1]
DevaForm : [1, 0]
Devotion : [1, 1]
EmptyFist : [2, 1]
EmptyMind : [4, 1]
Enlightenment : [1, 0]
Eruption : [3, 3]
Establishment : [1, 1]
Evaluate : [0, 1]
Fasting2 : [1, 0]
FearNoEvil : [2, 1]
Flash of Steel : [1, 0]
FlurryOfBlows : [2, 0]
FlyingSleeves : [1, 1]
ForeignInfluence : [1, 0]
Halt : [1, 0]
Indignation : [2, 0]
Injury : [1, 0]
InnerPeace : [2, 0]
LessonLearned : [3, 0]
Meditate : [2, 0]
MentalFortress : [2, 1]
Necronomicurse : [1, 0]
Nirvana : [1, 0]
Omniscience : [1, 0]
Pray : [1, 2]
Prostrate : [3, 1]
Protect : [3, 0]
Ragnarok : [2, 2]
ReachHeaven : [1, 0]
Sanctity : [3, 0]
SandsOfTime :

In [ ]:
#@title 사후 작업
write_wb.save("Run_data.xlsx")